## Concept extraction.

Concept extraction service aims to extract higher-level, human-friendly concepts from a trained neural network model instead of giving importance to input features (e.g., assigning importance weightings to pixels). For example, an understandable explanation of why an image classifier outputs the label "zebra" would ideally relate to concepts such as "stripes" rather than a set of particular pixel values.

Similarly, consider a classifier trained on images of firefighters. Now if the training data consisted mostly of males wearing slacks and helmets, then the model would assume that being male with a helmet was an important factor to be a firefighter. How would this help us? Well, this would bring out the bias in the training data which has fewer images of females and we could easily rectify that. We can then rank the training data concerning the ‘male’ concept and use that to balance the dataset better.

The Concept Extraction tool provides an interpretation of a neural net’s internal state in terms of human-friendly concepts. Concept Extraction uses directional derivatives to quantify the degree to which a user-defined concept is important to a classification result–for example, how sensitive a prediction of “zebra” is to the presence of stripes.

For concept extraction, we need at least 50 clear examples of concepts that you are looking for and a model which is likely to have learned the relevant concept. A quick way to figure this out (has the model learned a concept) would be to look at feature attribution/signal estimation for some images concerning that model. The examples should be either direct textures (for dependency on texture properties) or shapes (which should be clearly defined with multiple colors) or entire sub-objects (which should be cropped from the image). Given all of this information, our concept extraction framework finds out the internal part of the neural network which represents this sub-concept and uses that part’s activation output vector to rank any additional or unseen images for presence/absence/likelihood of the concept in those images data set.

In [ ]:
import torch
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
from untangle import UntangleAI

Relevant imports for this tutorial.

## Step 0 Training a CNN for recognizing MNIST dataset

This step is optional. If you would like to train a CNN network to recognize MNIST dataset you can refer to [this tutorial](/tutorials/mnist_model_training) which trains a model for 25 epochs and saves the trained weights into lenet_mnist_model.h5.

Or you can download the trained weights from [here](https://untanglemodels.s3.amazonaws.com/lenet_mnist_model.h5).

Load the model from checkpointed weights

In [ ]:
class LeNet(torch.nn.Module):
    # TODO: This isn't really a LeNet, but we implement this to be
    #  consistent with the Evidential Deep Learning paper
    def __init__(self):
        super(LeNet, self).__init__()
        self.model = None

    def build(self):
        lenet_conv = []
        lenet_conv += [torch.nn.Conv2d(1,20, kernel_size=(5,5))]
        lenet_conv += [torch.nn.ReLU(inplace=True)]
        lenet_conv += [torch.nn.MaxPool2d(kernel_size=(2,2), stride=2)]
        lenet_conv += [torch.nn.Conv2d(20, 50, kernel_size=(5,5))]
        lenet_conv += [torch.nn.ReLU(inplace=True)]
        lenet_conv += [torch.nn.MaxPool2d(kernel_size=(2,2), stride=2)]

        lenet_dense = []
        lenet_dense += [torch.nn.Linear(4*4*50, 500)]
        lenet_dense += [torch.nn.ReLU(inplace=True)]
        lenet_dense += [torch.nn.Linear(500, 10)]

        self.features = torch.nn.Sequential(*lenet_conv)
        self.classifier = torch.nn.Sequential(*lenet_dense)

    def forward(self, input):
        output = self.features(input)
        output = output.view(input.shape[0], -1)
        # output = output.view(-1, 4*4*50)
        output = self.classifier(output)
        return(output)

In [ ]:
print("Loading model checkpoint...", end="", flush=True)
model = LeNet()
model.build()

module_path = os.path.dirname(os.path.realpath('.'))
model_ckpt_path = os.path.join(module_path, 'lenet_mnist_model.h5')


if (torch.cuda.is_available()):
    ckpt = torch.load(model_ckpt_path)
    model.load_state_dict(ckpt)
    model = model.cuda()
else:
    ckpt = torch.load(model_ckpt_path, map_location='cpu')
    model.load_state_dict(ckpt)
    
print(model)

Create UntangleAI object and define helper functions to generate concepts and to display them.

In [ ]:
untangle_ai = UntangleAI()

In [ ]:
def generate_concepts(class_i, num_of_concepts, rotate_by_90deg = False):
    loader, _ = untangle_ai.load_mnist_per_class(batch_size=1, data_class=class_i)
    loader = list(loader) 
    concept_idx = random.sample(range(len(loader)), num_of_concepts)
    concepts = []
    for rn in concept_idx:
        tensor =  loader[rn][0]
        if rotate_by_90deg:
            tensor = tensor.permute(0, 1, 3, 2)
        concepts.append((class_i + '-' + str(rn), tensor))
    random.shuffle(concepts)   
    return concepts

In [ ]:
def show_concept_images(data, rows, fig_size=None):
    figure = plt.figure(figsize=fig_size)
    num_of_images = len(data)
    for index in range(1, num_of_images+1):
        plt.subplot(rows, num_of_images/rows, index)
        plt.axis('off')
        plt.imshow(data[index-1][1].numpy().squeeze(), cmap='gray_r')
    plt.show()

Let us try out our first concept, which is circle. In order to extract 50 images of circles, we just pick a random 50 samples from handwritten digit 0 and use that as concept for circle.

In [ ]:
concepts = generate_concepts('0', 50)
show_concept_images(concepts, 5, None)

Define a helper function to batch the data so that the operations can be vectorized and optimized for throughput on GPUs.

In [ ]:
def batchify_data(data, batch_size):
    img_path, tensors = zip(*data)
    batch = []
    for i in range(0, len(data), batch_size):
        batch.append((list(img_path[i : i+batch_size]), torch.cat(tensors[i : i+batch_size], dim=0)))

    return batch

In [ ]:
concept_batch  = batchify_data(concepts, 16)

Let us generate 1000 unseen data points to rank. We randomly sample them from the 0-9 digit classes.

In [ ]:
unseen_data = []
for i in range(10):
    unseen_data += generate_concepts(str(i), 100)
random.shuffle(unseen_data)
show_concept_images(unseen_data, 25, (16,16))

Auxilary data structure to retrive images from ranked list later.

In [ ]:
unseen_data_dict = {}
for img_path, tensor in unseen_data:
    unseen_data_dict[img_path] = tensor

Make batch of the unseen data which is used for ranking.

In [ ]:
unseen_data_batch = batchify_data(unseen_data, 64)

Define arguments that is used for learning and ranking concepts and which is passed to untangle API.

In [ ]:
class Args:
    mname = 'lenet5'
    act_type = 'flatten'
    train_data_path = './mnistasjpg/Training_Patterns_0/' # provide path containing concept images
    test_data_path = './mnistasjpg/trainingSample/' # probide path containing test images
    save_path = 'circle_concept_' # created ranked csv file with this name
    batch_size = 16
    img_size = (1,28,28)
args = Args()

Now call the `extract_concept_images` API to learn circle concept and rank unseen data.

In [ ]:
untangle_ai.extract_concept_images(model, concept_batch, unseen_data_batch, args)

Define a helper function to retrieve topk and bottomk ranked images based on the concept learnt.

In [ ]:
def get_top_and_bottom_ranks(filename, topk=20):
    df = pd.read_csv(filename)
    top20 = []
    bottom20 = []
    for i in range(topk):
        top20.append((df['image_name'].values[i], unseen_data_dict[df['image_name'].values[i]]))
        bottom20.append((df['image_name'].values[-1*(i+1)], unseen_data_dict[df['image_name'].values[-1*(i+1)]]))
    
    return (top20, bottom20)

In [ ]:
top20, bottom20 = get_top_and_bottom_ranks(args.save_path + 'ranked_list.csv')
show_concept_images(top20, 5)
show_concept_images(bottom20, 5)

As expected 0's, 6's, and 9's which have cricle as concept in their handwritten digit are top ranked and 4's, 7's, 5's and 1's which lack concept of circle are bottom ranked.

Now let us try with another new concpet which is horizontal line.

In [ ]:
concepts = generate_concepts('1', 75, True)
show_concept_images(concepts, 5, None)
concept_batch  = batchify_data(concepts, 16)
args.save_path = 'horizontal_line_'
untangle_ai.extract_concept_images(model, concept_batch, unseen_data_batch, args)
top20, bottom20 = get_top_and_bottom_ranks(args.save_path + 'ranked_list.csv')
show_concept_images(top20, 5)
show_concept_images(bottom20, 5)

As expected, 4s 5s 7s and 1s which have concept of horizontal line are top ranked, 0s. 3s. 2s, 6s and 9s which lack horizontal line concept are bottom ranked.

Let us try with one last concept of vertical lines. For this we will sample random images from hand written digit class 1

In [ ]:
concepts = generate_concepts('1', 50)
show_concept_images(concepts, 5, None)
concept_batch  = batchify_data(concepts, 16)
args.save_path = 'vertical_line_'
untangle_ai.extract_concept_images(model, concept_batch, unseen_data_batch, args)
top20, bottom20 = get_top_and_bottom_ranks(args.save_path + 'ranked_list.csv')
show_concept_images(top20, 5)
show_concept_images(bottom20, 5)

As expected we have 1s, 9s are top ranked which have concept of vertical line in their handwritten digit representation and the remaining classes are bottom ranked.

## Conclusion
Results are very sensitive to the concept images chosen. Handpicked images that are strong representative of concept would improve ranking of unseen data that have high likelihood of having that concept. 
Also more sophesticated and well trained models that have higher concept sensitivity than a simple model.

All the results are stored in csv format, which and be used for further analysis to derive recall rates.